In [ ]:
import numpy
import pygetm

%matplotlib widget
import matplotlib.pyplot

In [ ]:
halo = 2
Lx, Ly = 100., 100.

domain = pygetm.domain.Domain.create_cartesian(numpy.linspace(-Lx/2, Lx/2, 100), numpy.linspace(-Ly/2, Ly/2, 101), 1, H=1., lat=0., mask=0)
domain.mask[3:-3, 3:-3] = 1
domain.initialize(runtype=1)

# Mask out outer *2* rows and columns to ensure transports use a non-divergent flow field
domain.T.mask[...] = 0
domain.T.mask[2:-2, 2:-2] = 1

In [ ]:
period = 600
omega = 2 * numpy.pi / period
cfl = 1.
umax = omega * Lx / 2
dt_cfl = cfl * min(Lx / (domain.T.H.shape[1] + 1), Ly / (domain.T.H.shape[0] + 1)) / umax
no_of_revolutions = 5
Nmax = no_of_revolutions * round(2 * numpy.pi / omega / dt_cfl)
tmax = no_of_revolutions * 2 * numpy.pi / omega
timestep = tmax / Nmax

In [ ]:
u = domain.T.array(fill=0.)
v = domain.T.array(fill=0.)
var = domain.T.array(fill=1.)
u[:, :] = -omega * domain.U.y
v[:, :] = omega * domain.V.x
u[domain.U.mask == 0] = 0.
v[domain.V.mask == 0] = 0.
var[int(0.2 * domain.shape[1]):int(0.4 * domain.shape[1]), int(0.2 * domain.shape[2]):int(0.4 * domain.shape[2])] = 5.

In [ ]:
fig = matplotlib.pyplot.figure()
ax = fig.gca()
pc = ax.quiver(u[::5,::5],v[::5,::5])

In [ ]:
matplotlib.pyplot.ioff()
fig = matplotlib.pyplot.figure()
ax = fig.gca()
pc = ax.pcolormesh(var.ma)
cb = fig.colorbar(pc)
fig.show()

In [ ]:
adv = pygetm.Advection(domain, scheme=1)
for i in range(1, Nmax):
    print(var.values)
    adv.calculate(u, v, timestep, var)
    pc.set_array(var.ma.ravel())
    fig.canvas.draw()